In [193]:
from collections import OrderedDict
from collections import Counter
import numpy as np
import pandas
import heapq

In [194]:
visit_popularity = Counter()
purchase_popularity = Counter()
with open('train.txt', 'r') as f:
    for line in f.xreadlines():
        visits, purchases = line.strip().split(';')
        v=visits.split(',')
        for x in v:
            visit_popularity[x]+=1
        if purchases=='':
            continue
        else:   
            p=purchases.split(',')
            for t in p:
                purchase_popularity[t]+=1
#vis = {}
#purc={}
#for key, value in visit_popularity.items():
#    vis[key] = value
#for key, value in purchase_popularity.items():
#    purc[key] = value
#print vis['4']
#print purc


In [75]:
#with open('lol.txt', 'r') as f:
#    visits, purchases = f.readline().strip().split(';')
#p=purchases.split(',')
#print p
#for t in p:
#    purchase_popularity[p]+=1
#print visit_popularity      

In [195]:
def recommend_by_purchase(items, max_count):
    return heapq.nlargest(max_count, OrderedDict.fromkeys(items), key=lambda x: purchase_popularity.get(x, 0))

def recommend_by_visit(items, max_count):
    return heapq.nlargest(max_count, OrderedDict.fromkeys(items), key=lambda x: visit_popularity.get(x, 0))

In [113]:
recommend_by_visit(l,5)

['4', '2', '3', '0', '1']

In [146]:
#def sortbydictvisit(x):
#    return vis[x]
#l=['0','1','2','3','4','5']
#l.sort(key=sortbydictvisit,reverse=True)
#print l
#A=['1','2','3','9','6']
#B=['0','1','2','3','4','5']
#result=list(set(A) & set(B))
#a=len(result)
#print float(a)
#w=np.zeros(5)
#t=float(4)
#for i in range(5):
#    w[i]+=t
#print w
recommend_by_purchase()

[ 4.  4.  4.  4.  4.]


In [192]:
#avg_pre=np.zeros(5)
#avg_r=np.zeros(5)
#q='59,60,61,62,60,63,64,65,66,61,67,68,67;67,60,63'
#visits, purchases = q.strip().split(';')
#visits = visits.split(',')
#purchases = purchases.split(',')
#rec =recommend_by_purchase(visits,5)
#for k in range (5):
#obshie=len(list(set(rec[:5])& set(purchases)))
#t=float(obshie)
#avg_pre[4]+=t/(4+1)
#avg_r[4]+=t/len(purchases)
#print avg_r

                  
                    #avg_precision[k]+=t/(k+1)
                    #avg_recall[k]+=(t/len(purchases))
                    #sessions_count+=1

[ 0.  0.  0.  0.  1.]


In [196]:
def calculate_metrics(recommend, max_count, sessions_file):
    with open(sessions_file, 'r') as f:
        avg_recall = np.zeros(max_count)
        avg_precision = np.zeros(max_count)
        sessions_count = 0
        for line in f.xreadlines():
            visits, purchases = line.strip().split(';')
            if purchases != '':
                visits = visits.split(',')
                purchases = purchases.split(',')
                rec = recommend(visits, max_count)
                for k in range (max_count):
                    obshie=len(list(set(rec[:k+1])& set(purchases)))
                    t=float(obshie)
                    avg_precision[k]+=t/(k+1)
                    avg_recall[k]+=(t/len(purchases))
                    sessions_count+=1
                # TODO
                # update avg_recall
                # update avg_precision
                # update sessions_count
    return pandas.DataFrame({
            'k': np.arange(max_count) + 1,
            'avg_recall@k': [round(x, 2) for x in avg_recall / sessions_count],
            'avg_precision@k': [round(x, 2) for x in avg_precision / sessions_count]
    }).set_index('k')


In [197]:
calculate_metrics(recommend_by_purchase, 5, 'train.txt')

,avg_precision@k,avg_recall@k
k,,
1,0.16,0.14
2,0.11,0.17
3,0.08,0.18
4,0.06,0.18
5,0.05,0.19


In [198]:
calculate_metrics(recommend_by_purchase, 5, 'test.txt')

,avg_precision@k,avg_recall@k
k,,
1,0.11,0.09
2,0.08,0.13
3,0.06,0.15
4,0.05,0.16
5,0.04,0.16


In [199]:
calculate_metrics(recommend_by_visit, 5, 'train.txt')

,avg_precision@k,avg_recall@k
k,,
1,0.10,0.09
2,0.08,0.13
3,0.06,0.15
4,0.05,0.16
5,0.04,0.16


In [200]:
calculate_metrics(recommend_by_visit, 5, 'test.txt')

,avg_precision@k,avg_recall@k
k,,
1,0.10,0.08
2,0.07,0.12
3,0.06,0.14
4,0.05,0.15
5,0.04,0.16
